# ts_df

En este notebook se concatenan los datos originales con la predicción para poder hacer la comparativa entre los datos reales y los datos predichos.

## Librerias

In [ ]:
# imports time series
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Datasets

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/creation/Task 1/gb_union_weeks.csv')

In [ ]:
df.head()

In [ ]:
df['revenue_real']= df['sell_price'] * df['quantity']

In [ ]:
df.rename(columns={'quantity': 'quantity_real'}, inplace=True)

In [ ]:
ts=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/creation/Task 3/time_series_xgboost.csv')

In [ ]:
ts.head()

In [ ]:
ts['revenue_pred']= ts['sell_price'] * ts['quantity']

In [ ]:
ts.rename(columns={'quantity': 'quantity_pred'}, inplace=True)

## ts + df

Se unen todas las columnas del dataset original con la cantidad predicha

In [ ]:
ts_prepro = ts[['item', 'year_week', 'quantity_pred','store','region', 'revenue_pred']]
ts_prepro

In [ ]:
df_ts= pd.merge(df, ts_prepro, on=['item','year_week', 'store',	'region'], how='left')

In [ ]:
df_ts['porcent_diff'] = np.where(
    df_ts['quantity_pred'] != 0,  # Condicional para evitar la división por cero
    abs((df_ts['quantity_real'] - df_ts['quantity_pred']) / df_ts['quantity_pred'] * 100),
    np.nan  # Puedes elegir qué valor asignar cuando la predicción es cero, por ejemplo, NaN
)

In [ ]:
df_ts.tail()

In [ ]:
df_ts.head(5)

In [ ]:
df_ts.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/df_ts_ok.csv', index=False)

## quantity_real vs quantity_pred + revenue


In [ ]:

# Encuentra las últimas cuatro 'year_week' únicas
ultimas_cuatro_semanas = ts['year_week'].drop_duplicates().nlargest(4)

# Filtra el DataFrame para solo incluir las últimas cuatro semanas
df_ultimas_cuatro_semanas = ts[ts['year_week'].isin(ultimas_cuatro_semanas)]

# Agrupa por 'item' y suma quantity
predicciones_agrupadas = df_ultimas_cuatro_semanas.groupby('item')['quantity_pred'].sum().reset_index()

In [ ]:
predicciones_agrupadas

In [ ]:
last4weeks = df['year_week'].drop_duplicates().nlargest(4)

df_last4weeks = df[df['year_week'].isin(last4weeks)]

datos_reales = df_last4weeks.groupby(['item', 'category', 'department', 'sell_price'])['quantity_real'].sum().reset_index()


In [ ]:
datos_reales

In [ ]:
ts_item= pd.merge(datos_reales, predicciones_agrupadas, on='item', how='inner')

In [ ]:
ts_item

In [ ]:
ts_item['revenue_real'] = ts_item['quantity_real'] * ts_item['sell_price']
ts_item['revenue_pred'] = ts_item['quantity_pred'] * ts_item['sell_price']

In [ ]:
ts_item['porcent_diff'] = np.where(
    ts_item['quantity_pred'] != 0,  # Condicional para evitar la división por cero
    abs((ts_item['quantity_real'] - ts_item['quantity_pred']) / ts_item['quantity_pred']*100),
    np.nan  # Puedes elegir qué valor asignar cuando la predicción es cero, por ejemplo, NaN
)


In [ ]:
ts_item

In [ ]:
ts_item[ts_item['porcent_diff']==0.0]

In [ ]:
ts_item['porcent_diff'].max()

In [ ]:
ts_item[ts_item['porcent_diff']>100]

In [ ]:
ts_item.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/ts_item3.csv', index=False)

## quantity_real vs quantity_pred



In [ ]:

# Encuentra las últimas cuatro 'year_week' únicas
ultimas_cuatro_semanas = ts['year_week'].drop_duplicates().nlargest(4)

# Filtra el DataFrame para solo incluir las últimas cuatro semanas
df_ultimas_cuatro_semanas = ts[ts['year_week'].isin(ultimas_cuatro_semanas)]

# Agrupa por 'item' y suma quantity
predicciones_agrupadas = df_ultimas_cuatro_semanas.groupby('item')['quantity_pred'].sum().reset_index()

In [ ]:
predicciones_agrupadas

In [ ]:
last4weeks = df['year_week'].drop_duplicates().nlargest(4)

df_last4weeks = df[df['year_week'].isin(last4weeks)]

datos_reales = df_last4weeks.groupby(['item', 'category', 'department'])['quantity_real'].sum().reset_index()


In [ ]:
datos_reales

In [ ]:
ts_item= pd.merge(datos_reales, predicciones_agrupadas, on='item', how='inner')

In [ ]:
ts_item

In [ ]:
ts_item['porcent_diff'] = np.where(
    ts_item['quantity_pred'] != 0,  # Condicional para evitar la división por cero
    abs((ts_item['quantity_real'] - ts_item['quantity_pred']) / ts_item['quantity_pred']*100),
    np.nan  # Puedes elegir qué valor asignar cuando la predicción es cero, por ejemplo, NaN
)


In [ ]:
ts_item

In [ ]:
ts_item[ts_item['porcent_diff']==0.0]

In [ ]:
ts_item['porcent_diff'].max()

In [ ]:
ts_item[ts_item['porcent_diff']>100]

In [ ]:
ts_item.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/ts_item2.csv', index=False)

##**ITEMS: INTERANUAL GROWTH** (2013-2015)

In [ ]:
ts_item.shape

df

In [ ]:
df.head(5)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

In [ ]:
df.head()

In [ ]:
df_trends = pd.DataFrame(df.groupby(['item','year','department'])['quantity_real'].sum().reset_index())

In [ ]:
df_trends[df_trends['item'].duplicated()]

In [ ]:
#DATAFRAME
column_to_drop = df_trends[(df_trends['year'] == 2011) | (df_trends['year'] == 2016)].index
df_trends = df_trends.drop(column_to_drop)
df_trends.reset_index(drop=True, inplace=True)

In [ ]:
#LAGS
df_trends['quantity_real_lag1'] = df_trends['quantity_real'].shift(1)
df_trends['quantity_real_lag2'] = df_trends['quantity_real'].shift(2)

In [ ]:
df_trends_def = df_trends[df_trends['year']==2015]

In [ ]:
#Eliminate values 0
df_trends_def = df_trends_def[(df_trends_def['quantity_real_lag1'] != 0) & (df_trends_def['quantity_real_lag2'] != 0)]

#Eliminate column 'year'
del(df_trends_def['year'])

In [ ]:
df_trends_def.head()

In [ ]:
df_trends_def['Trends'] = df_trends_def['quantity_real']/((df_trends_def['quantity_real_lag1']+df_trends_def['quantity_real_lag2'])/2)
df_trends_def['Trends_pon'] = df_trends_def['quantity_real']/(((df_trends_def['quantity_real_lag1']*0.65)+(df_trends_def['quantity_real_lag2']*0.35)))

In [ ]:
df_trends_def.sort_values(by='Trends', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

# Definir los intervalos deseados
interval_size = 0.005

# Crear el histograma con los intervalos especificados
plt.figure(figsize=(10, 6))
#plt.hist(df_trends_def['Trends'], bins=int(1/interval_size), color='b', edgecolor='black')
plt.hist(df_trends_def['Trends_pon'], bins=int(1/interval_size), color='red', edgecolor='black')

# Mostrar la gráfica
plt.grid(True)
plt.tight_layout()
plt.show()
